# **Solving SVM Dual - Question 2**

## **Library**

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from cvxopt import matrix, solvers
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## **Data Preprocessing**

In [ ]:


# Replace 'your_file_id_here' with the actual file ID
file_id = '18I76FlBk2DIli3NbvnvPrG4nqZUETCO1'
# Read the CSV file into a DataFrame'
download_url = f'https://drive.google.com/uc?id={file_id}'


df = pd.read_csv(download_url)

In [ ]:
X = df.iloc[:, :-1].values  # last column is the label
y = df.iloc[:, -1].values   # labels

In [ ]:
# Convert labels: 0 to -1 and 1 to 1
y = np.where(y == 0, -1, 1)
X_train , X_test , y_train , y_test = train_test_split(X , y, test_size = 0.3)

array([[ 1.1197387 ,  0.009475  ,  0.2638998 , ...,  0.23404728,
         0.00697374,  1.4517994 ],
       [ 2.453599  , -0.14742422,  0.28554168, ...,  0.5980694 ,
        -0.05288332,  3.6111503 ],
       [ 2.1931965 , -0.13986062,  0.29031608, ...,  0.56720823,
        -0.02771215,  3.1775737 ],
       ...,
       [-0.8989769 ,  0.0766169 , -0.39390913, ..., -0.6827197 ,
        -0.32803258, -0.87315875],
       [ 0.27684978,  0.07935488,  0.19934243, ...,  0.01578434,
         0.05318773,  0.07524591],
       [-0.23955972,  0.03159761, -0.157269  , ..., -0.25828475,
         0.17565508, -0.70011437]])

## **Kernel Functions**

In [ ]:


def gaussian_kernel_matrix(X1, X2, gamma):
    """Compute the Gaussian (RBF) kernel matrix between two sets of vectors."""
    X1_sq = np.sum(X1 ** 2, axis=1)
    X2_sq = np.sum(X2 ** 2, axis=1)
    K = np.exp(-gamma * (np.outer(X1_sq, np.ones(X2.shape[0])) + np.outer(np.ones(X1.shape[0]), X2_sq) - 2 * np.dot(X1, X2.T)))
    return K

def polynomial_kernel_matrix(X1, X2, p):
    """Compute the Polynomial kernel matrix between two sets of vectors."""
    return (np.dot(X1, X2.T) + 1) ** p


## **Solving SVM dual using cvxopt**

In [ ]:
# SVM Dual Problem using CVXOPT
def fit_svm(X, y, C, kernel_func, gamma=None, p=None):
    n_samples, n_features = X.shape

    # Compute the kernel matrix using the selected kernel function
    if kernel_func == 'gaussian':
        K = gaussian_kernel_matrix(X, X, gamma)
    elif kernel_func == 'polynomial':
        K = polynomial_kernel_matrix(X, X, p)
    else:
        raise ValueError("Unsupported kernel function. Use 'gaussian' or 'polynomial'.")

    # Set up the parameters for the quadratic programming problem
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones(n_samples))
    G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = matrix(y, (1, n_samples), 'd')
    b = matrix(0.0)

    # Solve QP problem using CVXOPT
    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)

    alphas = np.ravel(solution['x'])
    return alphas


## **SVM cross validation for fine_tunning**

In [ ]:


def cross_validate_svm(X, y, C_values, gamma_values, p_values, kernel_func, k_folds=5):

    start_time = time.time()
    best_score = 0
    best_params = None

    for C in C_values:
        for gamma in gamma_values:
            for p in p_values:
                kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
                scores = []

                for train_index, test_index in kf.split(X):
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                    alphas = fit_svm(X_train, y_train, C, kernel_func, gamma, p)

                    # Extract support vectors
                    support_vectors = alphas > 1e-5
                    sv_alphas = alphas[support_vectors]
                    sv_X = X_train[support_vectors]
                    sv_y = y_train[support_vectors]

                    # Calculate bias term b using the support vectors
                    if kernel_func == 'gaussian':
                        K_sv = gaussian_kernel_matrix(sv_X, sv_X, gamma)
                    else:
                        K_sv = polynomial_kernel_matrix(sv_X, sv_X, p)

                    b = np.mean(sv_y - np.sum(sv_alphas * sv_y[:, np.newaxis] * K_sv, axis=0))

                    # Debugging output
                    print(f"Support Vectors Count: {len(sv_X)}")
                    print(f"Bias Term: {b}")

                    # Predict on the test set
                    if kernel_func == 'gaussian':
                        K_test = gaussian_kernel_matrix(X_test, sv_X, gamma)
                    elif kernel_func == 'polynomial':
                        K_test = polynomial_kernel_matrix(X_test, sv_X, p)
                    else:
                        raise ValueError("Unsupported kernel function. Use 'gaussian' or 'polynomial'.")

                    # Calculate decision values
                    decision_values = np.dot(K_test, sv_alphas * sv_y) + b
                    predictions = np.sign(decision_values)

                    accuracy = accuracy_score(y_test, predictions)
                    scores.append(accuracy)

                avg_score = np.mean(scores)
                print(f"Average Score for C={C}, gamma={gamma}, p={p}: {avg_score}")

                if avg_score > best_score:
                    best_score = avg_score
                    best_params = (C, gamma, p)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution Time: {execution_time} seconds")
    return best_params, best_score

# Define hyperparameter ranges
C_values = [0.1, 1, 10 ]
gamma_values = [0.1, 0.5, 1]
p_values = [2, 3, 5]

# Choose the kernel type
kernel_type = 'gaussian'  # or 'polynomial'

# Perform cross-validation
best_params, best_score = cross_validate_svm(X_train, y_train, C_values, gamma_values, p_values, kernel_type)
print('\n')
print('-----------------------------------------------------------------------------')
print("Best Hyperparameters (C, gamma, p):", best_params)
print("Best Cross-Validation Accuracy:", best_score)



Support Vectors Count: 225
Bias Term: -0.06141373799505384
Support Vectors Count: 241
Bias Term: -0.12207357036784174
Support Vectors Count: 180
Bias Term: -0.05950946434825987
Support Vectors Count: 192
Bias Term: -0.06274276601976365
Support Vectors Count: 198
Bias Term: -0.15535689601269087
Average Score for C=0.1, gamma=0.1, p=2: 0.9171428571428573
Support Vectors Count: 225
Bias Term: -0.06141373799505384
Support Vectors Count: 241
Bias Term: -0.12207357036784174
Support Vectors Count: 180
Bias Term: -0.05950946434825987
Support Vectors Count: 192
Bias Term: -0.06274276601976365
Support Vectors Count: 198
Bias Term: -0.15535689601269087
Average Score for C=0.1, gamma=0.1, p=3: 0.9171428571428573
Support Vectors Count: 225
Bias Term: -0.06141373799505384
Support Vectors Count: 241
Bias Term: -0.12207357036784174
Support Vectors Count: 180
Bias Term: -0.05950946434825987
Support Vectors Count: 192
Bias Term: -0.06274276601976365
Support Vectors Count: 198
Bias Term: -0.1553568960126

## **Predicition and accuracy**

In [ ]:
# Use the best hyperparameters to train the final SVM model
C_best, gamma_best, p_best = best_params
alphas_best = fit_svm(X_train, y_train, C_best, kernel_type, gamma_best, p_best)

# Extract support vectors (used only for bias calculation and prediction)
support_vectors = alphas_best > 1e-5
sv_alphas = alphas_best[support_vectors]
sv_X = X_train[support_vectors]
sv_y = y_train[support_vectors]

# Calculate the final bias term b using only the support vectors
if kernel_type == 'gaussian':
    K_sv = gaussian_kernel_matrix(sv_X, sv_X, gamma_best)
else:
    K_sv = polynomial_kernel_matrix(sv_X, sv_X, p_best)

b_best = np.mean(sv_y - np.sum(sv_alphas * sv_y[:, np.newaxis] * K_sv, axis=0))

# Print the optimal bias term and hyperparameters
print("Optimal bias term:", b_best)
print(f'Best params: C = {C_best}, gamma = {gamma_best}, p = {p_best}')

# Calculate the value of the objective function using the full set of training alphas
if kernel_type == 'gaussian':
    K_all = gaussian_kernel_matrix(X_train, X_train, gamma_best)
else:
    K_all = polynomial_kernel_matrix(X_train, X_train, p_best)

# Objective function calculation: sum of all alphas minus 1/2 * sum of all pairs of (alpha_i * alpha_j * y_i * y_j * K(x_i, x_j))
objective_value = np.sum(alphas_best) - 0.5 * np.sum(
    (alphas_best[:, np.newaxis] * alphas_best) * (y_train[:, np.newaxis] * y_train) * K_all
)

# Print the objective function value
print(f"Objective function value: {objective_value}")


# Define the prediction function
def prediction(X, sv_X, sv_y, sv_alphas, b_best, gamma_best, p_best, kernel_func='gaussian'):
    """
    Predict the labels for a dataset using the trained SVM model.

    Parameters:
    - X: np.array, the dataset (either train or test set) features
    - sv_X: np.array, support vectors
    - sv_y: np.array, labels of the support vectors
    - sv_alphas: np.array, alphas of the support vectors
    - b_best: float, bias term from training
    - gamma_best: float, best gamma value for Gaussian kernel
    - p_best: int, best degree for Polynomial kernel
    - kernel_func: str, type of kernel ('gaussian' or 'polynomial')

    Returns:
    - predictions: np.array, predicted labels
    """

    if kernel_func == 'gaussian':
        K_test = gaussian_kernel_matrix(X, sv_X, gamma_best)
    elif kernel_func == 'polynomial':
        K_test = polynomial_kernel_matrix(X, sv_X, p_best)
    else:
        raise ValueError("Unsupported kernel function. Use 'gaussian' or 'polynomial'.")

    # Compute decision values
    decision_values = np.dot(K_test, sv_alphas * sv_y) + b_best
    predictions = np.sign(decision_values)

    return predictions

# Make predictions on train and test set using the final model
y_train_pred = prediction(X_train, sv_X, sv_y, sv_alphas, b_best, gamma_best, p_best, kernel_func=kernel_type)
y_test_pred = prediction(X_test, sv_X, sv_y, sv_alphas, b_best, gamma_best, p_best, kernel_func=kernel_type)

# Evaluate and print the accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
print()
print(f"Training Set Accuracy: {train_accuracy:.2f}")
print(f"Test Set Accuracy: {test_accuracy:.2f}")


# Confusion matrices
conf_matrix_train = confusion_matrix(y_train, y_train_pred)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)

print("Confusion Matrix (Training Set):")
print(conf_matrix_train)
print()
print("Confusion Matrix (Test Set):")
print(conf_matrix_test)


Optimal bias term: -0.7868726923034077
Best params: C = 1, gamma = 0.5, p = 2
Objective function value: 126.15176987746685

Training Set Accuracy: 0.92
Test Set Accuracy: 0.92
Confusion Matrix (Training Set):
[[327  27]
 [ 30 316]]

Confusion Matrix (Test Set):
[[134  12]
 [ 13 141]]


# **SMO algorithm - Question 3**

## **gaussian kernel**

In [ ]:

# Gaussian kernel function
def gaussian_kernel(X1, X2, gamma):
    if X1.ndim == 1 and X2.ndim == 1:
        return np.exp(-gamma * np.linalg.norm(X1 - X2) ** 2)
    elif X1.ndim > 1 and X2.ndim == 1:
        return np.exp(-gamma * np.linalg.norm(X1 - X2, axis=1) ** 2)
    elif X1.ndim > 1 and X2.ndim > 1:
        return np.exp(-gamma * np.linalg.norm(X1[:, np.newaxis] - X2[np.newaxis, :], axis=2) ** 2)


## **Objective function**

In [ ]:
# Compute objective function
def compute_objective_function(alpha, y, K):
    return np.sum(alpha) - 0.5 * np.dot(alpha * y, np.dot(K, alpha * y))


## **SMO algorithm**

In [ ]:




# SMO algorithm with MVP
def smo_svm(X, y, C, gamma, tol=1e-5, max_passes=10):
    n_samples = X.shape[0]
    alpha = np.zeros(n_samples)
    b = 0.0
    passes = 0
    total_iterations = 0

    # Precompute the Kernel matrix
    K = gaussian_kernel(X, X, gamma)

    # Error cache
    E = np.zeros(n_samples)

    while passes < max_passes:
        num_changed_alphas =0


        for i in range(n_samples):
            # Calculate E[i]
            total_iterations += 1
            E[i] = np.sum((alpha * y) * K[:, i]) + b - y[i]

            if (y[i]*E[i] < -tol and alpha[i] < C) or (y[i]*E[i] > tol and alpha[i] > 0):
                # Select j (Most Violating Pair)
                j = np.argmax(np.abs(E - E[i]))
                if i == j:
                    continue

                # Save old alphas
                alpha_i_old = alpha[i]
                alpha_j_old = alpha[j]

                # Compute L and H
                if y[i] != y[j]:
                    L = max(0, alpha[j] - alpha[i])
                    H = min(C, C + alpha[j] - alpha[i])
                else:
                    L = max(0, alpha[i] + alpha[j] - C)
                    H = min(C, alpha[i] + alpha[j])
                if L == H:
                    continue

                # Compute eta
                eta = 2.0 * K[i, j] - K[i, i] - K[j, j]
                if eta >= 0:
                    continue

                # Update alpha[j]
                alpha[j] -= y[j] * (E[i] - E[j]) / eta
                # Clip alpha[j]
                alpha[j] = np.clip(alpha[j], L, H)

                # Check for significant change
                if abs(alpha[j] - alpha_j_old) < tol:
                    continue

                # Update alpha[i]
                alpha[i] += y[i]*y[j]*(alpha_j_old - alpha[j])

                # Compute b1 and b2
                b1 = b - E[i] - y[i]*(alpha[i] - alpha_i_old)*K[i, i] - y[j]*(alpha[j] - alpha_j_old)*K[i, j]
                b2 = b - E[j] - y[i]*(alpha[i] - alpha_i_old)*K[i, j] - y[j]*(alpha[j] - alpha_j_old)*K[j, j]

                # Update b
                if 0 < alpha[i] < C:
                    b = b1
                elif 0 < alpha[j] < C:
                    b = b2
                else:
                    b = (b1 + b2) / 2.0

                num_changed_alphas += 1

        if num_changed_alphas == 0:
            passes += 1
        else:
            passes = 0

    # Compute the final objective function value
    obj = compute_objective_function(alpha, y, K)

    # Count iterations (approximate)
    num_iterations = passes

    return alpha, b, obj, total_iterations

# Set hyperparameters
C = 10.0
gamma = 0.5
tol = 1e-5
max_passes = 10

# Compute initial objective function value
n_samples_train = X_train.shape[0]
K_train = gaussian_kernel(X_train, X_train, gamma)
alpha_init = np.zeros(n_samples_train)
obj_init = compute_objective_function(alpha_init, y_train, K_train)
print(f"Initial Objective Function Value: {obj_init:.4f}")

# Train the SVM
alpha, b, obj_final, num_iterations = smo_svm(X_train, y_train, C, gamma, tol, max_passes)
print(f"Final Objective Function Value: {obj_final:.4f}")
print(f"Number of Iterations (Passes): {num_iterations}")
print(f"Number of Support Vectors: {np.sum(alpha > 0)}")






Initial Objective Function Value: 0.0000
Final Objective Function Value: 326.9523
Number of Iterations (Passes): 15400
Number of Support Vectors: 70


## **Prediction**

In [ ]:
# Prediction function
def predict(X_train, y_train, X_test, alpha, b, gamma):
    K = gaussian_kernel(X_test, X_train, gamma)
    return np.sign(np.dot(K, alpha * y_train) + b)

# Evaluate performance
y_train_pred = predict(X_train, y_train, X_train, alpha, b, gamma)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

y_test_pred = predict(X_train, y_train, X_test, alpha, b, gamma)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Accuracy: 91.00%
Test Accuracy: 90.33%


## **Hyperparameter Tunning**

In [ ]:
# Example hyperparameter tuning (simplified)
start_time = time.time()

best_accuracy = 0
best_C = None
best_gamma = None

for gamma in [0.1, 0.5, 1.0]:
    for C in [0.1, 1.0, 10.0]:
        alpha, b, obj_final, num_iterations = smo_svm(X_train, y_train, C, gamma, tol, max_passes)
        y_val_pred = predict(X_train, y_train, X_test, alpha, b, gamma)
        val_accuracy = accuracy_score(y_test, y_val_pred)
        print(f"Gamma: {gamma}, C: {C}, Validation Accuracy: {val_accuracy * 100:.2f}%")
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_C = C
            best_gamma = gamma

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")
print()

print(f"Best Validation Accuracy: {best_accuracy * 100:.2f}% with C={best_C} and gamma={best_gamma}")


Gamma: 0.1, C: 0.1, Validation Accuracy: 66.67%
Gamma: 0.1, C: 1.0, Validation Accuracy: 80.67%
Gamma: 0.1, C: 10.0, Validation Accuracy: 74.33%
Gamma: 0.5, C: 0.1, Validation Accuracy: 67.33%
Gamma: 0.5, C: 1.0, Validation Accuracy: 70.00%
Gamma: 0.5, C: 10.0, Validation Accuracy: 90.33%
Gamma: 1.0, C: 0.1, Validation Accuracy: 75.00%
Gamma: 1.0, C: 1.0, Validation Accuracy: 69.33%
Gamma: 1.0, C: 10.0, Validation Accuracy: 69.33%
Execution Time: 6.03749942779541 seconds

Best Validation Accuracy: 90.33% with C=10.0 and gamma=0.5


# **Ethnicity Clasification**

## **Data Preprocessing**

In [ ]:

# Replace 'your_file_id_here' with the actual file ID
file_id = '1zffI34PfHh-27JdvUWN779ICYVaiXcuZ'
# Read the CSV file into a DataFrame'
download_url = f'https://drive.google.com/uc?id={file_id}'


df = pd.read_csv(download_url)

In [ ]:
classes_to_keep = [0 , 1 , 2]

df = df[df['gt'].isin(classes_to_keep)]

X = df.iloc[:, :-1].values  # last column is the label
y = df.iloc[:, -1].values   # labels

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y, test_size = 0.3)

## **Gaussian kernell**

In [ ]:
# Define the Gaussian kernel function
def gaussian_kernel(X1, X2, gamma):
    if X1.ndim == 1:
        X1 = X1[np.newaxis, :]
    if X2.ndim == 1:
        X2 = X2[np.newaxis, :]
    sq_dists = np.sum(X1**2, axis=1).reshape(-1, 1) + \
               np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
    return np.exp(-gamma * sq_dists)

## **Training svm**

In [ ]:

# Define the SVM training function with Gaussian kernel
def train_svm(X, y, C=1.0, gamma=0.05):

    n_samples = X.shape[0]
    K = gaussian_kernel(X, X, gamma)
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones(n_samples))
    G_std = np.diag(-np.ones(n_samples))
    h_std = np.zeros(n_samples)
    G_slack = np.diag(np.ones(n_samples))
    h_slack = C * np.ones(n_samples)
    G = matrix(np.vstack((G_std, G_slack)))
    h = matrix(np.hstack((h_std, h_slack)))
    A = matrix(y.reshape(1, -1).astype('double'))
    b = matrix(np.zeros(1))
    solvers.options['show_progress'] = False
    start_time = time.time()


    solution = solvers.qp(P, q, G, h, A, b)


    end_time = time.time()
    training_time = end_time - start_time


    alphas = np.ravel(solution['x'])
    sv = alphas > 1e-5
    ind = np.arange(len(alphas))[sv]
    alphas_sv = alphas[sv]
    X_sv = X[sv]
    y_sv = y[sv]
    b_value = 0


    for n in range(len(alphas_sv)):
        b_value += y_sv[n]
        b_value -= np.sum(alphas_sv * y_sv * K[ind[n], sv])
    b_value /= len(alphas_sv)
    iterations = solution['iterations']
    objective = solution['primal objective']


    model = {
        'alphas': alphas_sv,
        'X_sv': X_sv,
        'y_sv': y_sv,
        'b': b_value,
        'C': C,
        'gamma': gamma,
        'training_time': training_time,
        'iterations': iterations,
        'objective': objective
    }


    return model




## **Prediction and Accuracy**

In [ ]:
# Train the SVM classifiers with Gaussian kernel
classes = np.unique(y_train)
models = {}
C = 1.0  # Regularization parameter
gamma = 0.5  # Gaussian kernel hyperparameter

for c in classes:
    print(f"Training classifier for class {c} vs. rest")
    y_binary = np.where(y_train == c, 1, -1)
    model = train_svm(X_train, y_binary, C=C, gamma=gamma)
    models[c] = model

# Define prediction functions
def project(model, X):
    K = gaussian_kernel(X, model['X_sv'], model['gamma'])
    y_predict = np.sum(model['alphas'] * model['y_sv'] * K, axis=1) + model['b']
    return y_predict

def predict(models, X):
    predictions = np.zeros((X.shape[0], len(models)))
    classes = list(models.keys())
    for i, c in enumerate(classes):
        model = models[c]
        y_predict = project(model, X)
        predictions[:, i] = y_predict
    class_indices = np.argmax(predictions, axis=1)
    y_pred = [classes[i] for i in class_indices]
    return np.array(y_pred)

# Make predictions
y_train_pred = predict(models, X_train)
y_test_pred = predict(models, X_test)

# Evaluate performance
def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"\nTraining accuracy: {train_accuracy * 100:.2f}%")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

def confusion_matrix(y_true, y_pred, labels):
    n_labels = len(labels)
    cm = np.zeros((n_labels, n_labels), dtype=int)
    label_to_index = {label: index for index, label in enumerate(labels)}
    for t, p in zip(y_true, y_pred):
        i = label_to_index[t]
        j = label_to_index[p]
        cm[i, j] += 1
    return cm

labels = classes
cm = confusion_matrix(y_test, y_test_pred, labels)

print("\nConfusion Matrix:")
print(cm)

# Report metrics
print(f"\nHyperparameters:")
print(f"C (Regularization parameter): {C}")
print(f"Gamma (Gaussian kernel parameter): {gamma}")
print("Kernel: Gaussian (RBF)")

total_training_time = sum(model['training_time'] for model in models.values())
print(f"Total training time: {total_training_time:.2f} seconds")

total_iterations = sum(model['iterations'] for model in models.values())
print(f"Total number of optimization iterations: {total_iterations}")

for c, model in models.items():
    print(f"Class {c} SVM dual objective value: {model['objective']}")

Training classifier for class 0 vs. rest
Training classifier for class 1 vs. rest
Training classifier for class 2 vs. rest

Training accuracy: 94.76%
Test accuracy: 89.56%

Confusion Matrix:
[[137   9  10]
 [  8 140   7]
 [  8   5 126]]

Hyperparameters:
C (Regularization parameter): 1.0
Gamma (Gaussian kernel parameter): 0.5
Kernel: Gaussian (RBF)
Total training time: 8.05 seconds
Total number of optimization iterations: 33
Class 0 SVM dual objective value: -167.67525784070426
Class 1 SVM dual objective value: -136.66189889939014
Class 2 SVM dual objective value: -152.0289143906552
